# Anime : FFHQ-Alignment

---


- [`Happy-jihye`](https://github.com/happy-jihye)
- [`happy-jihye/GAN`](https://github.com/happy-jihye/GAN)

### Reference

- **landmark detector** : 
  1. [`1adrianb/face-alignment`](https://github.com/1adrianb/face-alignment) : `pip install face-alignment`
  2. [`nagadomi/lbpcascade_animeface`](https://github.com/nagadomi/lbpcascade_animeface)
  3. [`kanosawa/anime_face_landmark_detection`](https://github.com/kanosawa/anime_face_landmark_detection)
  
  
- **FFHQ alignment** : [NVlabs/ffhq-dataset](https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py)
  

In [1]:
import os
import sys

import numpy as np
import scipy.ndimage
import PIL.Image
import face_alignment

import matplotlib.pyplot as plt
from skimage import io
import imageio
import natsort

## 1. Using `face-alignment`

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
image_path = 'raw_image'
image_path_list = [os.path.join(image_path, im) for im in os.listdir(image_path) if im.endswith('.jpg')]

### 1.1 FFHQ-Alignment & Save Images

In [5]:
from ffhq_align import image_align_68

landmark_path = f'align_image'
os.makedirs(landmark_path, exist_ok=True)

landmarks_detector = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)
print(f'total : {len(image_path_list)}')

error = []
i = 0

for image_path in image_path_list:
    face_landmarks = landmarks_detector.get_landmarks(image_path)
    
    if face_landmarks is None:
        error.append(image_path)
        continue        

    aligned_face_path = os.path.join(landmark_path, f'align-{str(i).zfill(4)}.png')

    image_align_68(image_path, aligned_face_path, face_landmarks[0])
    i+=1

print(f'success : {i}')
print(f'fail : {len(image_path_list) - i}')

total : 7


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/conda/lib/python3.8/site-packages/face_alignment/api.py:132: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")


success : 4
fail : 3


### 1.2 Write Errors

In [6]:
txt = open(f'error.txt', 'w')
for er in error:
    txt.write(er + '\n')
txt.close()

### 1.3 (Option) Save images with the landmark

In [7]:
landmark_path = f'landmark'

os.makedirs(landmark_path, exist_ok=True)

print(f'total : {len(image_path_list)}')
i = 0

for image_path in image_path_list:
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)

    input = io.imread(image_path)
    preds = fa.get_landmarks(input)

    det = fa.get_landmarks_from_image(image_path)
    plt.imshow(input) 
    
    if det == None:
        plt.close()
        continue
    
    for detection in det:
        scat = plt.scatter(detection[:,0], detection[:,1], 2)

    plt.xticks([])
    plt.yticks([])
    plt.savefig(landmark_path + f'/landmark-{str(i).zfill(4)}.png', bbox_inches='tight', pad_inches=0)
    scat.remove()
    plt.close()
    i += 1
    
print(f'success : {i}')
print(f'fail : {len(image_path_list) - i}')

total : 7
success : 4
fail : 3


## 2. Using `face-alignment`

In [8]:
import numpy as np
import torch
from torchvision import transforms
import cv2
from PIL import Image, ImageDraw
from ffhq_align import CFA, image_align_24

### 2.1 Download the cascade file and checkpoint

In [48]:
!wget https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml

--2021-08-04 16:01:32--  https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246945 (241K) [text/plain]
Saving to: ‘lbpcascade_animeface.xml’

lbpcascade_animefac 100%[===================>] 241.16K  --.-KB/s    in 0.1s    

2021-08-04 16:01:32 (2.28 MB/s) - ‘lbpcascade_animeface.xml’ saved [246945/246945]



In [58]:
drive_id = '1NckKw7elDjQTllRxttO87WY7cnQwdMqz'

url = f'https://drive.google.com/file/d/{drive_id}/view' 
id= url.split('/')[-2]
!gdown --id $id 

Downloading...
From: https://drive.google.com/uc?id=1NckKw7elDjQTllRxttO87WY7cnQwdMqz
To: /workspace/notebooks/FFHQ-Alignmnet/checkpoint_landmark_191116.pth.tar
16.7MB [00:00, 37.8MB/s]


### 2.2 Read Errors of 1

In [9]:
f = open(f'error.txt', 'r')

error = []
while True:
    line = f.readline()
    if not line: 
        break   
    error.append(line.strip('\n'))

### 2.3 FFHQ-Alignment & Save Images

In [10]:
# path
landmark_path = f'align_iamge'
os.makedirs(landmark_path, exist_ok=True)

# param
num_landmark = 24
img_width = 256
checkpoint_name = 'checkpoint_landmark_191116.pth.tar'

# detector
face_detector = cv2.CascadeClassifier('lbpcascade_animeface.xml')
landmark_detector = CFA(output_channel_num=num_landmark + 1, checkpoint_name=checkpoint_name).cuda()

# transform
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5])
train_transform = [transforms.ToTensor(), normalize]
train_transform = transforms.Compose(train_transform)

error2 = []
k = 0
for error_image in error:

    # input image & detect face
    input_img_name = error_image
    img = cv2.imread(input_img_name)
    faces = face_detector.detectMultiScale(img)

    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    if faces == ():
        error2.append(error_image)
        continue

    for x_, y_, w_, h_ in faces:

        # adjust face size
        x = max(x_ - w_ / 8, 0)
        rx = min(x_ + w_ * 9 / 8, img.width)
        y = max(y_ - h_ / 4, 0)
        by = y_ + h_
        w = rx - x
        h = by - y


        # transform image
        img_tmp = img.crop((x, y, x+w, y+h))
        img_tmp = img_tmp.resize((img_width, img_width), Image.BICUBIC)

        img_tmp_tf = train_transform(img_tmp)
        img_tmp_tf = img_tmp_tf.unsqueeze(0).cuda()

        # estimate heatmap
        heatmaps = landmark_detector(img_tmp_tf)
        heatmaps = heatmaps[-1].cpu().detach().numpy()[0]

        # landmark array
        landmark_array = []

        # calculate landmark position
        for i in range(num_landmark):
            heatmaps_tmp = cv2.resize(heatmaps[i], (img_width, img_width), interpolation=cv2.INTER_CUBIC)
            landmark = np.unravel_index(np.argmax(heatmaps_tmp), heatmaps_tmp.shape)
            landmark_y = landmark[0] 
            landmark_x = landmark[1] 

            # draw landmarks
            #draw.ellipse((landmark_x - 2, landmark_y - 2, landmark_x + 2, landmark_y + 2), fill=(255, 0, 0))
            landmark_array.append([landmark_x - 2, landmark_y - 2, landmark_x + 2, landmark_y + 2])

    # output image
    #img_tmp.save(f'./LineWebtoonCharacterDataset/MyDeepestSecret/Emma/align2-{str(k).zfill(4)}.png')

    image_align_24(img_tmp, f'{landmark_path}/align2-{str(k).zfill(4)}.png',landmark_array , transform_size=256 )
    k+=1

<ipython-input-10-49b5d9d6da77>:31: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


### 2.4 Crop Images

In [12]:
image_path = 'raw_image'
image_path_list = [os.path.join(image_path, im) for im in os.listdir(image_path) if im.endswith('.jpg')]

In [13]:
# path
crop_path = f'crop'
os.makedirs(crop_path, exist_ok=True)

# param
img_width = 256

# detector
face_detector = cv2.CascadeClassifier('lbpcascade_animeface.xml')

k = 0
for image_path in image_path_list:

    # input image & detect face
    img = cv2.imread(image_path)
    faces = face_detector.detectMultiScale(img)

    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    if faces == ():
        continue

    for x_, y_, w_, h_ in faces:

        # adjust face size
        x = max(x_ - w_ / 8, 0)
        rx = min(x_ + w_ * 9 / 8, img.width)
        y = max(y_ - h_ / 4, 0)
        by = y_ + h_
        w = rx - x
        h = by - y

        # crop image
        img = img.crop((x, y, x+w, y+h))
        img = img.resize((img_width, img_width), Image.BICUBIC)

    # output image
    img.save(f'{crop_path}/crop-{str(k).zfill(4)}.png')

    k+=1

<ipython-input-13-6f40fbf7c57d>:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


### 2.4 (Option) Save images with the landmark

In [160]:
import numpy as np
import torch
from torchvision import transforms
import cv2
from PIL import Image, ImageDraw
from ffhq_align import CFA, image_align_24

# path
landmark_path = f'./LineWebtoonCharacterDataset/{webtoon}/{name}/landmark'
os.makedirs(landmark_path, exist_ok=True)

# param
num_landmark = 24
img_width = 256
checkpoint_name = 'checkpoint_landmark_191116.pth.tar'

# detector
face_detector = cv2.CascadeClassifier('lbpcascade_animeface.xml')
landmark_detector = CFA(output_channel_num=num_landmark + 1, checkpoint_name=checkpoint_name).cuda()

# transform
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5])
train_transform = [transforms.ToTensor(), normalize]
train_transform = transforms.Compose(train_transform)

error2 = []
print(f'total : {len(error)}')
k = 0
for error_image in error:

    # input image & detect face
    input_img_name = error_image
    img = cv2.imread(input_img_name)
    faces = face_detector.detectMultiScale(img)

    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #draw = ImageDraw.Draw(img)

    if faces == ():
        error2.append(error_image)
        continue

    for x_, y_, w_, h_ in faces:

        # adjust face size
        x = max(x_ - w_ / 8, 0)
        rx = min(x_ + w_ * 9 / 8, img.width)
        y = max(y_ - h_ / 4, 0)
        by = y_ + h_
        w = rx - x
        h = by - y
        
        # transform image
        img_tmp = img.crop((x, y, x+w, y+h))
        img_tmp = img_tmp.resize((img_width, img_width), Image.BICUBIC)
        
        draw = ImageDraw.Draw(img_tmp)

        img_tmp_tf = train_transform(img_tmp)
        img_tmp_tf = img_tmp_tf.unsqueeze(0).cuda()

        # estimate heatmap
        heatmaps = landmark_detector(img_tmp_tf)
        heatmaps = heatmaps[-1].cpu().detach().numpy()[0]

        # calculate landmark position
        for i in range(num_landmark):
            heatmaps_tmp = cv2.resize(heatmaps[i], (img_width, img_width), interpolation=cv2.INTER_CUBIC)
            landmark = np.unravel_index(np.argmax(heatmaps_tmp), heatmaps_tmp.shape)
            landmark_y = landmark[0] 
            landmark_x = landmark[1] 

            # draw landmarks
            draw.ellipse((landmark_x - 2, landmark_y - 2, landmark_x + 2, landmark_y + 2), fill=(255, 0, 0))

    # output image
    img_tmp.save(f'{landmark_path}/landmark2-{str(k).zfill(4)}.png')

    k+=1

print(f'success : {k}')
print(f'fail : {len(error) - k}')

total : 114


<ipython-input-160-65edddf11f14>:40: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


success : 25
fail : 89


In [150]:
txt = open(f'./LineWebtoonCharacterDataset/{webtoon}/{name}/error2.txt', 'w')
for er in error2:
    txt.write(er + '\n')
txt.close()